In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [39]:
dataset_dir = './garbage_classification/'

In [40]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [41]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 12415 images belonging to 12 classes.


In [43]:
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_generator = val_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation split
)

Found 3100 images belonging to 12 classes.


In [48]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding="same",input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), padding="same", activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), padding="same", activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(12, activation='softmax')  # 6 categories
])

In [49]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 12)             │           396 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,412,524 (20.65 MB)

 Trainable params: 5,412,524 (20.65 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [51]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/20


/home/stonie/miniconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


387/387 ━━━━━━━━━━━━━━━━━━━━ 59s 142ms/step - accuracy: 0.3575 - loss: 2.0403 - val_accuracy: 0.4417 - val_loss: 1.6115
Epoch 2/20
  1/387 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.6250 - loss: 1.5288

/home/stonie/miniconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6250 - loss: 1.5288 - val_accuracy: 0.5357 - val_loss: 1.4298
Epoch 3/20


...

2024-11-24 13:10:07.225522: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14908659050251824735
2024-11-24 13:10:07.225577: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9219512066643145910


387/387 ━━━━━━━━━━━━━━━━━━━━ 52s 131ms/step - accuracy: 0.6346 - loss: 1.1482 - val_accuracy: 0.6432 - val_loss: 1.0636
Epoch 18/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 0s 50us/step - accuracy: 0.5938 - loss: 1.2983 - val_accuracy: 0.6071 - val_loss: 1.2142
Epoch 19/20


2024-11-24 13:10:58.808943: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14908659050251824735


387/387 ━━━━━━━━━━━━━━━━━━━━ 53s 134ms/step - accuracy: 0.6375 - loss: 1.1204 - val_accuracy: 0.6292 - val_loss: 1.1446
Epoch 20/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 0s 38us/step - accuracy: 0.6250 - loss: 1.1206 - val_accuracy: 0.5714 - val_loss: 1.4072


2024-11-24 13:11:51.718655: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14908659050251824735
2024-11-24 13:11:51.718692: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9219512066643145910


In [64]:
# Save the model
model.export('waste_sorting_model')


INFO:tensorflow:Assets written to: waste_sorting_model/assets


INFO:tensorflow:Assets written to: waste_sorting_model/assets


Saved artifact at 'waste_sorting_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor_60')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  131278317012944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131278317011216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619621008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619619856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619619472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619621200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619621392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619621968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619622160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131277619622736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1312776

In [65]:
converter = tf.lite.TFLiteConverter.from_saved_model('waste_sorting_model')
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('waste_sorting_model.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1732443789.613244    5344 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1732443789.613264    5344 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-11-24 13:23:09.613572: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: waste_sorting_model
2024-11-24 13:23:09.614741: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-24 13:23:09.614769: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: waste_sorting_model
2024-11-24 13:23:09.621667: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-11-24 13:23:09.623106: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-24 13:23:09.682780: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: waste_sorting_model
2024-11-24 13:23:09.695492: I tensorflow/cc/saved_model/

In [55]:
# Evaluate the model (if test data is separate)
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # You can use a separate test split if available
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")


Found 3100 images belonging to 12 classes.
 6/97 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.6389 - loss: 1.0342

/home/stonie/miniconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6434 - loss: 1.0777
Test Accuracy: 0.64


In [54]:
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Confusion matrix and classification report
print(confusion_matrix(y_true, y_pred_classes))
print(classification_report(y_true, y_pred_classes, target_names=val_generator.class_indices.keys()))

97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
[[ 12  14   3  11  76   8   6  30   8  13   6   2]
 [ 15   9   3   9  87  10   4  35   7  12   4   2]
 [  6   8   6   9  58   1   3  16   2  10   1   1]
 [ 15   9   3   9  75   4   5  31  11  10   3   3]
 [ 63  56  38  51 480  27  18 171  40  86  16  19]
 [  5  14   0   4  48   6   0  22   6  12   2   6]
 [  9   8   5   9  59   6   2  28   7  16   3   1]
 [ 16  10   6  12  85   6   6  31   7  20   2   9]
 [  9   8   3   7  74  10   8  31   6  15   1   1]
 [ 21  26  11  13 173  13   6  76  23  28   1   4]
 [  8   9   2   6  64   7   1  20   4  15   3   0]
 [  7   8   5   9  71   4   5  25   3  14   3   1]]
              precision    recall  f1-score   support

     battery       0.06      0.06      0.06       189
  biological       0.05      0.05      0.05       197
 brown-glass       0.07      0.05      0.06       121
   cardboard       0.06      0.05      0.06       178
     clothes       0.36      0.45      0.40      1065
 green-glass       0.0